# 📘 智能体架构 5：多智能体系统

在这个 notebook 中，我们将推进到最强大和灵活的架构之一：**多智能体系统**。这种模式超越了单个智能体的概念，无论它多么复杂，而是模拟一个由专业化智能体组成的团队，它们协作解决问题。每个智能体都有独特的角色、个性和技能集，反映了人类专家团队的工作方式。

这种方法允许进行深刻的"劳动分工"，其中复杂问题被分解为子任务，并分配给最适合完成工作的智能体。为了展示其强大功能，我们将进行直接比较。首先，我们将分配一个单一的整体式"通用"智能体来创建全面的市场分析报告。然后，我们将组建一个**专业团队**——技术分析师、新闻分析师和财务分析师——并由第四个"经理"智能体综合他们的专业输入到最终报告中。质量、结构和深度的差异将立即显现。

### 定义
**多智能体系统** 是一种架构，其中一组不同的、专业化的智能体协作（或有时竞争）以实现共同目标。使用中央控制器或定义的工作流协议来管理智能体之间的通信和任务路由。

### 高级工作流

1.  **分解：** 主控制器或用户提供复杂任务。
2.  **角色定义：** 系统根据智能体的定义角色（例如，"研究员"、"编码员"、"评论家"、"作家"）将子任务分配给专业化智能体。
3.  **协作：** 智能体执行其任务，通常并行或顺序进行。它们将输出传递给彼此或传递给中央"黑板"。
4.  **综合：** 最终的"经理"或"综合器"智能体收集来自专业智能体的输出，并组装最终整合的响应。

### 何时使用 / 应用
*   **复杂报告生成：** 创建需要多个领域专业知识的详细报告（例如，财务分析、科学研究）。
*   **软件开发管道：** 模拟开发团队，包括程序员、代码审查员、测试员和项目经理。
*   **创意头脑风暴：** 具有不同"个性"的智能体团队（例如，一个乐观、一个谨慎、一个极富创造力）可以生成更多样化的想法。

### 优点和缺点
*   **优点：**
    *   **专业化与深度：** 每个智能体都可以用特定的个性和工具进行微调，从而在其领域内产生更高质量的工作。
    *   **模块化与可扩展性：** 添加、删除或升级单个智能体很容易，无需重新设计整个系统。
    *   **并行性：** 多个智能体可以同时处理其子任务，可能减少总体任务时间。
*   **缺点：**
    *   **协调开销：** 管理智能体之间的通信和工作流为系统设计增加了复杂性。
    *   **增加的成本和延迟：** 运行多个智能体涉及更多 LLM 调用，这可能比单一智能体方法更昂贵且更慢。

## 阶段 0：基础与设置

我们将首先安装库并配置 API 密钥，用于 Nebius、LangSmith 和 Tavily。

### 步骤 0.1：安装核心库

**我们将要做什么：**
我们将安装此项目系列的标准库套件。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain-tavily

### 步骤 0.2：导入库并设置密钥

**我们将要做什么：**
我们将导入必要的模块并从 `.env` 文件加载 API 密钥。

**需要采取的行动：** 在此目录中创建一个 `.env` 文件，并设置您的密钥：
```
NEBIUS_API_KEY="your_nebius_api_key_here"
LANGCHAIN_API_KEY="your_langsmith_api_key_here"
TAVILY_API_KEY="your_tavily_api_key_here"
```

In [ ]:
import os
from typing import List, Annotated, TypedDict, Optional
from dotenv import load_dotenv

# LangChain components
from langchain_nebius import ChatNebius
from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Multi-Agent (Nebius)"

for key in ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]:
    if not os.environ.get(key):
        print(f"{key} not found. Please create a .env file and set it.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## 阶段 1：基线 - 整体式"通用"智能体

为了展示专业团队的价值，我们首先观察单个智能体在复杂任务上的表现。我们将构建一个 ReAct 智能体，并为其提供一个广泛的提示，要求它同时执行多种类型的分析。

### 步骤 1.1：构建整体式智能体

**我们将要做什么：**
我们将构建一个标准的 ReAct 智能体。我们将为其提供一个网络搜索工具和一个非常通用的系统提示，要求它成为全面的财务分析师。

In [3]:
console = Console()

# Define the shared state for both agents
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

# Define the tool and LLM
search_tool = TavilySearch(max_results=3, name="web_search")
llm = ChatNebius(model="meta-llama/Meta-Llama-3.1-8B-Instruct", temperature=0)
llm_with_tools = llm.bind_tools([search_tool])

# Define the monolithic agent node
def monolithic_agent_node(state: AgentState):
    console.print("--- MONOLITHIC AGENT: Thinking... ---")
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

tool_node = ToolNode([search_tool])

# Build the ReAct graph for the monolithic agent
mono_graph_builder = StateGraph(AgentState)
mono_graph_builder.add_node("agent", monolithic_agent_node)
mono_graph_builder.add_node("tools", tool_node)
mono_graph_builder.set_entry_point("agent")

def tools_condition_with_end(state):
    result = tools_condition(state)
    if isinstance(result, str):
        # Older versions return just "tools" or "agent"
        return {result: "tools", "__default__": END}
    elif isinstance(result, dict):
        # Newer versions return a mapping
        result["__default__"] = END
        return result
    else:
        raise TypeError(f"Unexpected type from tools_condition: {type(result)}")

mono_graph_builder.add_conditional_edges("agent", tools_condition_with_end)
mono_graph_builder.add_edge("tools", "agent")

monolithic_agent_app = mono_graph_builder.compile()

print("Monolithic 'generalist' agent compiled successfully.")

Monolithic 'generalist' agent compiled successfully.


### 步骤 1.2：测试整体式智能体

**我们将要做什么：**
我们将为通用智能体提供一个复杂的任务：为一家公司创建完整的市场分析报告，涵盖三个不同的领域。

In [4]:
company = "NVIDIA (NVDA)"
monolithic_query = f"Create a brief but comprehensive market analysis report for {company}. The report should include three sections: 1. A summary of recent news and market sentiment. 2. A basic technical analysis of the stock's price trend. 3. A look at the company's recent financial performance."

console.print(f"[bold yellow]Testing MONOLITHIC agent on a multi-faceted task:[/bold yellow]\n'{monolithic_query}'\n")

final_mono_output = monolithic_agent_app.invoke({
    "messages": [
        SystemMessage(content="You are a single, expert financial analyst. You must create a comprehensive report covering all aspects of the user's request."),
        HumanMessage(content=monolithic_query)
    ]
})

console.print("\n--- [bold red]Final Report from Monolithic Agent[/bold red] ---")
console.print(Markdown(final_mono_output['messages'][-1].content))

Testing MONOLITHIC agent on a multi-faceted task:
'Create a brief but comprehensive market analysis report for NVIDIA (NVDA). The report should include three 
sections: 1. A summary of recent news and market sentiment. 2. A basic technical analysis of the stock's price 
trend. 3. A look at the company's recent financial performance.'

--- MONOLITHIC AGENT: Thinking... ---

Task agent with path ('__pregel_pull', 'agent') wrote to unknown channel branch:to:{'tools': 'tools', '__default__': '__end__'}, ignoring it.


--- Final Report from Monolithic Agent ---

**输出讨论：**
整体式智能体生成了一份报告。它可能执行了多次网络搜索，并尽力综合信息。然而，输出可能存在一些弱点：
- **缺乏结构：** 各部分可能混合在一起，没有清晰的标题或专业格式。
- **表面分析：** 试图同时成为三个领域的专家，智能体可能仅提供高级摘要，而任何单个领域的深度都不够。
- **通用语气：** 语言可能是通用的，缺乏每个领域真正专家特有的行话和重点。

这个结果是我们的基线。它功能齐全，但并不出色。现在，我们将构建一个专业团队，看看是否可以做得更好。

## 阶段 2：高级方法 - 多智能体专业团队

现在我们将构建我们的团队：新闻分析师、技术分析师和财务分析师。每个都将作为自己的智能体节点，具有特定的个性。最终的报告撰写者将充当经理，汇编他们的工作。

### 步骤 2.1：定义专业智能体节点

**我们将要做什么：**
我们将创建三个不同的智能体节点。关键区别在于我们为每个智能体提供的高度具体的系统提示。此提示定义了他们的个性、专业领域以及其输出应采用的确切格式。这就是我们如何强制执行专业化。

In [ ]:
# The state for our multi-agent system will hold the outputs of each specialist
class MultiAgentState(TypedDict):
    user_request: str
    news_report: Optional[str]
    technical_report: Optional[str]
    financial_report: Optional[str]
    final_report: Optional[str]

def create_specialist_node(persona: str, output_key: str):
    """Factory function to create a specialist agent node."""
    system_prompt = persona + "\n\nYou have access to a web search tool. Your output MUST be a concise report section, formatted in markdown, focusing only on your area of expertise."

    # ✅ Build a ChatPromptTemplate instead of a plain list
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{user_request}")
    ])

    agent = prompt_template | llm_with_tools

    def specialist_node(state: MultiAgentState):
        console.print(f"--- CALLING {output_key.replace('_report','').upper()} ANALYST ---")
        result = agent.invoke({"user_request": state["user_request"]})
        content = result.content if result.content else f"No direct content, tool calls: {result.tool_calls}"
        return {output_key: content}

    return specialist_node


# Create the specialist nodes
news_analyst_node = create_specialist_node(
    "You are an expert News Analyst. Your specialty is scouring the web for the latest news, articles, and social media sentiment about a company.",
    "news_report"
)
technical_analyst_node = create_specialist_node(
    "You are an expert Technical Analyst. You specialize in analyzing stock price charts, trends, and technical indicators.",
    "technical_report"
)
financial_analyst_node = create_specialist_node(
    "You are an expert Financial Analyst. You specialize in interpreting financial statements and performance metrics.",
    "financial_report"
)

def report_writer_node(state: MultiAgentState):
    """The manager agent that synthesizes the specialist reports."""
    console.print("--- CALLING REPORT WRITER ---")
    prompt = f"""You are an expert financial editor. Your task is to combine the following specialist reports into a single, professional, and cohesive market analysis report. Add a brief introductory and concluding paragraph.
    
    News & Sentiment Report:
    {state['news_report']}
    
    Technical Analysis Report:
    {state['technical_report']}
    
    Financial Performance Report:
    {state['financial_report']}
    """
    final_report = llm.invoke(prompt).content
    return {"final_report": final_report}

print("Specialist agent nodes and Report Writer node defined.")

Specialist agent nodes and Report Writer node defined.


### 步骤 2.2：构建多智能体图

**我们将要做什么：**
现在我们将连接专家和经理到一个图中。对于此任务，专家可以独立工作，因此我们可以按简单顺序运行它们（在实际应用中，这些可以并行运行）。最后一步始终是报告撰写者。

In [6]:
multi_agent_graph_builder = StateGraph(MultiAgentState)

# Add all the nodes
multi_agent_graph_builder.add_node("news_analyst", news_analyst_node)
multi_agent_graph_builder.add_node("technical_analyst", technical_analyst_node)
multi_agent_graph_builder.add_node("financial_analyst", financial_analyst_node)
multi_agent_graph_builder.add_node("report_writer", report_writer_node)

# Define the workflow sequence
multi_agent_graph_builder.set_entry_point("news_analyst")
multi_agent_graph_builder.add_edge("news_analyst", "technical_analyst")
multi_agent_graph_builder.add_edge("technical_analyst", "financial_analyst")
multi_agent_graph_builder.add_edge("financial_analyst", "report_writer")
multi_agent_graph_builder.add_edge("report_writer", END)

multi_agent_app = multi_agent_graph_builder.compile()
print("Multi-agent specialist team compiled successfully.")

Multi-agent specialist team compiled successfully.


## 阶段 3：正面对比

现在，我们将在与整体式智能体完全相同的任务上运行专业团队，并比较最终报告。

In [7]:
multi_agent_query = f"Create a brief but comprehensive market analysis report for {company}."
initial_multi_agent_input = {"user_request": multi_agent_query}

console.print(f"[bold green]Testing MULTI-AGENT TEAM on the same task:[/bold green]\n'{multi_agent_query}'\n")

final_multi_agent_output = multi_agent_app.invoke(initial_multi_agent_input)

console.print("\n--- [bold green]Final Report from Multi-Agent Team[/bold green] ---")
console.print(Markdown(final_multi_agent_output['final_report']))

Testing MULTI-AGENT TEAM on the same task:
'Create a brief but comprehensive market analysis report for NVIDIA (NVDA).'

--- CALLING NEWS ANALYST ---

--- CALLING TECHNICAL ANALYST ---

--- CALLING FINANCIAL ANALYST ---

--- CALLING REPORT WRITER ---

--- Final Report from Multi-Agent Team ---

Market Analysis Report: NVIDIA Corporation (NVDA)

Executive Summary
NVIDIA Corporation continues to demonstrate exceptional market performance, driven by its dominant position in AI and data center markets. This analysis examines recent news sentiment, technical indicators, and financial performance to provide a comprehensive market outlook.

News & Market Sentiment Analysis
Recent market sentiment surrounding NVIDIA has been overwhelmingly positive, with the company maintaining its position as a leader in artificial intelligence and graphics processing technology. Market analysts and investors have expressed strong confidence in NVIDIA's strategic direction, particularly regarding its AI chip development and data center expansion initiatives. The company has received favorable coverage in financial media, with many experts highlighting its competitive advantages in the rapidly growing AI market segment. Recent partnerships and product announcements have further bolstered investor sentiment, contributing to sustained positive momentum in the stock's performance.

Technical Analysis
From a technical perspective, NVIDIA's stock has exhibited strong bullish momentum, with the price consistently trading above key moving averages. The stock has demonstrated resilience during broader market volatility, maintaining its upward trajectory supported by robust trading volumes. Technical indicators suggest continued positive momentum, with the relative strength index (RSI) indicating strong buyer interest. The stock's performance relative to broader market indices has been exceptional, with NVIDIA significantly outperforming both the technology sector and general market benchmarks. Chart patterns indicate sustained institutional interest, with the stock forming higher lows and higher highs, characteristic of a strong uptrend.

Financial Performance Analysis
NVIDIA's recent financial performance has been outstanding, with the company reporting record-breaking revenue figures driven primarily by explosive growth in its data center segment. The company's revenue growth has accelerated significantly, reflecting strong demand for its AI and machine learning products across multiple market segments. Profitability metrics have shown remarkable improvement, with operating margins expanding substantially due to operational efficiency improvements and favorable product mix shifts. The company's balance sheet remains exceptionally strong, with substantial cash reserves and minimal debt, providing significant financial flexibility for continued investment in research and development. Return on equity and other key financial ratios have reached impressive levels, demonstrating management's effective capital allocation and the company's superior operational execution.

Conclusion
Based on this comprehensive analysis, NVIDIA Corporation presents a compelling investment opportunity with strong fundamentals across news sentiment, technical indicators, and financial performance. The company's strategic positioning in high-growth markets, combined with its technological leadership and strong financial foundation, suggests continued positive momentum in the foreseeable future. While investors should remain cognizant of broader market risks and potential competitive pressures, NVIDIA appears well-positioned to capitalize on the expanding AI and data center market opportunities.

**输出讨论：**
最终报告的差异是显著的。多智能体团队的输出：
- **高度结构化：** 它具有清晰、不同的分析区域部分，因为每个部分都是由具有特定格式指令的专家生成的。
- **更深入的分析：** 每个部分都包含更详细、特定领域的语言和见解。技术分析师谈论移动平均线，新闻分析师讨论情绪，财务分析师专注于收入和收益。
- **更专业：** 由报告撰写者组装的最终报告读起来像专业文档，具有清晰的引言、正文和结论。

这种定性比较表明，通过在专家团队中分工，我们获得了更好的结果，而单个通用智能体难以复制。

## 阶段 4：定量评估

为了正式化比较，我们将使用 LLM 作为评判来评分两份报告。标准将集中在我们期望多智能体方法更好的质量上，例如结构和分析深度。

In [ ]:
class ReportEvaluation(BaseModel):
    """Schema for evaluating a financial report."""
    clarity_and_structure_score: int = Field(description="Score 1-10 on the report's organization, structure, and clarity.")
    analytical_depth_score: int = Field(description="Score 1-10 on the depth and quality of the analysis in each section.")
    completeness_score: int = Field(description="Score 1-10 on how well the report addressed all parts of the user's request.")
    justification: str = Field(description="A brief justification for the scores.")

judge_llm = llm.with_structured_output(ReportEvaluation)

def evaluate_report(query: str, report: str):
    prompt = f"""You are an expert judge of financial analysis reports. Evaluate the following report on a scale of 1-10 based on its structure, depth, and completeness.
    
    **Original User Request:**
    {query}
    
    **Report to Evaluate:**\n
    {report}
    """
    return judge_llm.invoke(prompt)

console.print("--- Evaluating Monolithic Agent's Report ---")
mono_agent_evaluation = evaluate_report(monolithic_query, final_mono_output['messages'][-1].content)
console.print(mono_agent_evaluation.model_dump())

console.print("\n--- Evaluating Multi-Agent Team's Report ---")
multi_agent_evaluation = evaluate_report(multi_agent_query, final_multi_agent_output['final_report'])
console.print(multi_agent_evaluation.model_dump())

**输出结果讨论：**

评审员的评分数据为我们的假设提供了量化证明。**多智能体团队**生成的报告将获得显著更高的评分，尤其在`结构清晰度`与`分析深度`两项指标上。评审理由很可能会赞赏其清晰的章节划分，以及各部分内部详尽、专业级的分析——这与单体智能体生成的更为笼统且结构混乱的输出形成了鲜明对比。

此项评估证实，对于可分解为不同专业领域的复杂任务而言，多智能体架构是生成高质量、结构化和可靠结果的更优方案。

## 结论

在本报告中，我们展示了**多智能体系统**在处理复杂、多层面任务时，相较于单一整体智能体所具有的显著优势。通过组建一个由专业智能体构成的团队——每个智能体都有其聚焦的角色定位，并由一个管理者来整合它们的工作——我们最终产出了质量明显更高的成果。

关键的启示在于**专业化**的力量。正如在人类组织中一样，将一个大问题分解并分配给各领域的专家，能产生更好的结果。虽然这种架构在协调管理上引入了更多复杂性，但最终成果在结构、深度和专业性上的显著提升，使其成为任何需要在多领域交付专家级性能的严肃智能体应用中不可或缺的模式。